In [ ]:
import os
import pandas as pd
from concurrent.futures import ThreadPoolExecutor

# Specify the directory path
directory_path = '/home/mostafa2rz/Desktop/Tennis_project/202405'
# Specify the prefixes you want to filter files with
prefixes = ['away_team_1','away_team_score','event','home_team_1','home_team_score','round','season','time','tournament','venue','odds', 'pbp' ,'statistics' ,'power', 'votes']

# Function to read and concatenate Parquet files for a given prefix
def read_concat_parquet_files(prefix):
    filtered_files = [os.path.join(root, file) for root, _, files in os.walk(directory_path) for file in files if file.startswith(prefix)]
    df = [pd.read_parquet(file, engine='pyarrow') for file in filtered_files]
    result_df = pd.concat(df, ignore_index=True)
    
    # Save the DataFrame to a CSV file with the corresponding prefix
    result_df.to_csv(f'{directory_path}/df_{prefix}.csv', index=False)
    
    return prefix, result_df

# Dictionary to store DataFrames
dfs = {}

# Use ThreadPoolExecutor for parallel processing
with ThreadPoolExecutor() as executor:
    results = executor.map(read_concat_parquet_files, prefixes)

# Store the results in the dictionary
for prefix, result_df in results:
    key = f'df_{prefix}'  # Use the prefix as the key in the dictionary
    dfs[key] = result_df

# Access each DataFrame from the dictionary with the prefix as part of the key
for key in dfs:
    globals()[key] = dfs[key]
